# Análisis de Sentimiento con Modelos Preentrenados de HuggingFace

## Objetivo

En esta actividad vamos a utilizar un modelo de estado del arte ya entrenado para analizar el sentimiento de frases en español con apenas unas líneas de código, gracias a la librería HuggingFace Transformers.

Vamos a mostrar cómo es posible aprovechar el poder de los Transformers como BERT sin necesidad de entrenar redes neuronales desde cero.

### ¿Por qué esto es revolucionario?

Hasta ahora entrenamos modelos desde cero:
- Perceptrón con 6 ejemplos
- MLP con 10 ejemplos
- LSTM con 10 ejemplos

**El problema**: Con tan pocos datos, los modelos aprenden poco y generalizan mal.

**La solución**: Usar modelos que ya fueron entrenados con millones de textos. Esto se llama **transfer learning** (aprendizaje por transferencia).

## 1. Instalación de la librería transformers

Si estás en Google Colab, instalá la librería con el siguiente comando:

In [ ]:
!pip install -q transformers

## 2. ¿Qué es BETO?

Antes de usar el modelo, entendamos qué estamos cargando.

### BETO = BERT en Español

**BERT** (Bidirectional Encoder Representations from Transformers) es una arquitectura de modelo de lenguaje desarrollada por Google en 2018 que revolucionó el NLP.

**BETO** es la versión en español de BERT, entrenada específicamente con textos en español.

### Características de BETO:

- **Entrenado con**: Wikipedia en español (millones de artículos)
- **Arquitectura**: Transformer con atención bidireccional
- **Parámetros**: ~110 millones
- **Tiempo de entrenamiento**: Días en múltiples GPUs
- **Costo de entrenamiento**: Miles de dólares en recursos computacionales

### ¿Qué significa "preentrenado"?

BETO ya aprendió:
- Gramática del español
- Vocabulario extenso
- Relaciones semánticas entre palabras
- Contextos en los que aparecen las palabras

Nosotros vamos a usar ese conocimiento para análisis de sentimiento, sin tener que entrenar nada desde cero.

## 3. Cargando el modelo de análisis de sentimiento

Vamos a usar `finiteautomata/beto-sentiment-analysis`, que es BETO ya ajustado específicamente para clasificar sentimientos en español.

### ¿Qué es un pipeline?

Un **pipeline** en HuggingFace encapsula todo el proceso:
1. Tokenización del texto
2. Conversión a formato del modelo
3. Inferencia (predicción)
4. Post-procesamiento de resultados

Todo esto en una sola línea de código.

In [ ]:
###############################################################################
# CARGA DEL MODELO PREENTRENADO: BETO para Análisis de Sentimiento
###############################################################################

from transformers import pipeline

print("Cargando modelo BETO para análisis de sentimiento...")
print("Este proceso descarga el modelo (puede tardar unos segundos la primera vez)\n")

###############################################################################
# ¿Qué es un pipeline en HuggingFace?
###############################################################################
# Un pipeline es una abstracción que encapsula TODO el proceso de inferencia:
#
# 1. TOKENIZACIÓN:
#    - Convierte texto a tokens (sub-palabras)
#    - Ejemplo: "increíble" → ["in", "##creíble"] o ["increíble"]
#    - Agrega tokens especiales: [CLS] al inicio, [SEP] al final
#    - Convierte tokens a IDs numéricos según vocabulario del modelo
#
# 2. PREPARACIÓN DE ENTRADA:
#    - Crea tensors de PyTorch/TensorFlow
#    - Agrega padding si es necesario
#    - Crea attention masks (qué posiciones son padding, qué son reales)
#
# 3. INFERENCIA (PREDICCIÓN):
#    - Pasa los tensors por la red neuronal
#    - BETO procesa la secuencia con 12 capas de Transformers
#    - Cada capa aplica self-attention y feed-forward
#    - La última capa clasifica: positivo, negativo, o neutral
#
# 4. POST-PROCESAMIENTO:
#    - Convierte logits (números crudos) a probabilidades con softmax
#    - Selecciona la clase con mayor probabilidad
#    - Formatea resultado como diccionario: {'label': ..., 'score': ...}
#
# Todo esto pasa internamente cuando hacemos: clasificador("texto")
#
###############################################################################

###############################################################################
# Creación del pipeline con modelo específico
###############################################################################
# pipeline(tarea, model=nombre_del_modelo)
#
# Parámetro 1: "sentiment-analysis"
# Especifica la tarea de NLP que queremos realizar.
# HuggingFace soporta múltiples tareas:
# - "sentiment-analysis": Clasificación de sentimiento
# - "text-generation": Generación de texto (como GPT)
# - "question-answering": Responder preguntas sobre un contexto
# - "fill-mask": Completar palabras enmascaradas (como BERT original)
# - "ner": Named Entity Recognition (detectar nombres, lugares, etc.)
# - "translation": Traducción entre idiomas
# - "summarization": Resumir textos
# - y muchas más...
#
# Parámetro 2: model="finiteautomata/beto-sentiment-analysis"
# Especifica QUÉ modelo preentrenado usar.
#
# ¿Qué significa este nombre?
# - "finiteautomata": Organización que publicó el modelo en HuggingFace Hub
# - "beto-sentiment-analysis": Nombre del modelo específico
#
# ¿Qué es este modelo exactamente?
# Es BETO (BERT en español) que pasó por DOS etapas de entrenamiento:
#
# ETAPA 1: Preentrenamiento de BETO (hecho por dccuchile/beto)
#   - Corpus: Wikipedia en español (millones de artículos)
#   - Tarea: Masked Language Modeling (predecir palabras enmascaradas)
#   - Duración: Días en GPUs potentes
#   - Resultado: BETO entiende gramática, semántica, y contexto del español
#
# ETAPA 2: Fine-tuning para sentimiento (hecho por finiteautomata)
#   - Corpus: Dataset etiquetado con sentimientos (positivo/negativo/neutral)
#   - Tarea: Clasificación de sentimiento
#   - Duración: Horas/días
#   - Resultado: BETO ajustado específicamente para analizar sentimientos
#
# ¿Cuántos parámetros tiene este modelo?
# Aproximadamente 110 millones de parámetros:
# - 12 capas Transformer
# - 768 dimensiones de embedding
# - 12 attention heads por capa
# - Vocabulario de ~31,000 tokens
#
# Comparación:
# - Nuestro perceptrón: 17 parámetros
# - Nuestra MLP: 145 parámetros
# - Nuestra LSTM: ~7,000 parámetros
# - BETO: ~110,000,000 parámetros (15,000 veces más que LSTM!)
#
# ¿Dónde se descarga el modelo?
# Primera vez: Se descarga desde HuggingFace Hub (~400 MB)
# Se guarda en cache local: ~/.cache/huggingface/
# Próximas veces: Se carga desde cache (instantáneo)
#
# ¿Qué archivos se descargan?
# - config.json: Configuración del modelo (capas, dimensiones, etc.)
# - pytorch_model.bin o model.safetensors: Pesos del modelo (~400 MB)
# - vocab.txt: Vocabulario (lista de 31,000 tokens)
# - tokenizer_config.json: Configuración del tokenizador
# - special_tokens_map.json: Tokens especiales ([CLS], [SEP], etc.)
#
clasificador = pipeline("sentiment-analysis", model="finiteautomata/beto-sentiment-analysis")

###############################################################################
# ¿Qué contiene el objeto 'clasificador'?
###############################################################################
# Es un objeto Pipeline que internamente tiene:
#
# 1. clasificador.tokenizer: 
#    - Tokenizador específico de BETO
#    - Usa WordPiece tokenization (como BERT original)
#    - Convierte texto → tokens → IDs
#
# 2. clasificador.model:
#    - La red neuronal BETO completa
#    - 12 capas Transformer (encoder)
#    - Capa de clasificación al final (3 clases: POS, NEG, NEU)
#    - ~110 millones de parámetros ya entrenados
#
# 3. clasificador.postprocessor:
#    - Convierte salida cruda del modelo a formato amigable
#    - Aplica softmax para obtener probabilidades
#    - Mapea IDs de clases a etiquetas legibles
#
# ¿Cómo se usa?
# Simplemente: clasificador("texto a clasificar")
# Devuelve: [{'label': 'POS', 'score': 0.95}]
#
###############################################################################

print("Modelo cargado exitosamente")
print(f"Modelo: {clasificador.model.config._name_or_path}")
print(f"\nEste modelo fue ajustado (fine-tuned) específicamente para español.")

###############################################################################
# Información adicional sobre el modelo
###############################################################################
print("\nCaracterísticas técnicas de BETO:")
print(f"  - Arquitectura: Transformer (BERT-base)")
print(f"  - Parámetros: ~110 millones")
print(f"  - Capas: 12 capas de self-attention")
print(f"  - Dimensión de embeddings: 768")
print(f"  - Attention heads por capa: 12")
print(f"  - Vocabulario: ~31,000 tokens (WordPiece)")
print(f"  - Contexto máximo: 512 tokens")
print(f"  - Entrenado en: Wikipedia en español")
print(f"  - Fine-tuned para: Clasificación de sentimiento (3 clases)")

print("\n¿Por qué NO necesitamos entrenar?")
print("  1. El modelo YA aprendió español (preentrenamiento)")
print("  2. El modelo YA aprendió sentimientos (fine-tuning)")
print("  3. Solo necesitamos usarlo para inferencia")
print("  4. Esto se llama TRANSFER LEARNING")

print("\nVentajas de usar modelos preentrenados:")
print("  ✓ No necesitamos millones de ejemplos de entrenamiento")
print("  ✓ No necesitamos GPUs potentes ni días de entrenamiento")
print("  ✓ Obtenemos resultados de estado del arte inmediatamente")
print("  ✓ El modelo ya entiende lenguaje natural complejo")
print("  ✓ Podemos enfocarnos en la aplicación, no en el entrenamiento")

## 4. Comparación con nuestros modelos anteriores

Antes de probar BETO, recordemos las limitaciones de nuestros modelos:

| Aspecto | Nuestros modelos | BETO |
|---------|-----------------|------|
| Datos de entrenamiento | 6-10 frases | Millones de textos |
| Vocabulario | 10-50 palabras | ~30,000 tokens |
| Parámetros | Cientos/Miles | 110 millones |
| Tiempo de entrenamiento | Segundos | Días en GPUs |
| Comprensión del lenguaje | Básica | Avanzada |

Esta diferencia de escala es lo que hace que BETO sea tan poderoso.

## 5. Evaluación de frases variadas

Ahora vamos a probar el modelo con frases reales, incluyendo expresiones típicas de Argentina.

In [ ]:
###############################################################################
# USANDO EL CLASIFICADOR: ¿Qué pasa cuando predecimos?
###############################################################################

frases = [
    "Este lugar es espectacular, lo recomiendo totalmente",
    "Una decepción total. No pienso volver",
    "Más o menos... esperaba otra cosa",
    "Qué buena onda la atención, me encantó",
    "Mala calidad, pésimo servicio",
    "Zafa, pero nada especial",
    "Me sentí muy bien atendido",
    "Una estafa. Me arrepiento totalmente",
    "Todo excelente, 10 puntos",
    "Nunca más. Fue un desastre"
]

print("="*70)
print("PREDICCIONES DE BETO EN FRASES VARIADAS")
print("="*70)

###############################################################################
# ¿Qué hace clasificador(frases) internamente?
###############################################################################
# Paso a paso del proceso de inferencia:
#
# 1. TOKENIZACIÓN (para cada frase):
#    Ejemplo con "Este lugar es espectacular"
#    a. Separar en tokens: ["Este", "lugar", "es", "espectacular"]
#    b. Agregar tokens especiales: ["[CLS]", "Este", "lugar", "es", "espectacular", "[SEP]"]
#    c. Convertir a IDs: [101, 2156, 3421, 2098, 15678, 102]
#       donde 101=[CLS], 102=[SEP], y los demás son IDs del vocabulario
#
# 2. PREPARACIÓN DE TENSORS:
#    - Crear tensor de input_ids: [[101, 2156, 3421, 2098, 15678, 102], ...]
#    - Crear attention_mask: [[1, 1, 1, 1, 1, 1], ...] (1=real, 0=padding)
#    - Si las frases tienen diferente largo, se agrega padding al final
#
# 3. PROCESAMIENTO POR BETO (12 capas):
#
#    Capa de Embeddings:
#    - Token embeddings: Convierte cada ID a vector de 768 dimensiones
#    - Position embeddings: Agrega información de posición en la frase
#    - Segment embeddings: Para distinguir segmentos (no usado aquí)
#    - Suma: embedding_total = token_emb + pos_emb + seg_emb
#
#    Capas 1-12 (Transformer Encoder):
#    Cada capa hace:
#    
#    a. Multi-Head Self-Attention (12 heads):
#       - Cada token "presta atención" a todos los demás tokens
#       - Aprende qué palabras son importantes en el contexto
#       - Ejemplo: "no" presta mucha atención a "me gustó"
#       - Fórmula: Attention(Q,K,V) = softmax(Q·Kᵀ/√d)·V
#       - Q, K, V se calculan con matrices aprendidas
#    
#    b. Feed-Forward Network:
#       - Dos capas lineales con activación GELU
#       - Procesa cada posición independientemente
#       - Expande a 3072 dimensiones y vuelve a 768
#    
#    c. Normalización y conexiones residuales:
#       - Layer normalization después de cada sub-capa
#       - Conexiones skip para facilitar el entrenamiento
#
#    Output de las 12 capas:
#    - Representación contextual de cada token (768 dimensiones)
#    - El token [CLS] contiene una representación de toda la frase
#
# 4. CLASIFICACIÓN:
#    - Toma el vector del token [CLS] (768 dimensiones)
#    - Pasa por una capa Dense: [768] → [3]
#    - 3 salidas (logits) para: NEG, NEU, POS
#    - Aplica softmax: convierte logits a probabilidades
#    - Suma de probabilidades = 1.0
#
# 5. POST-PROCESAMIENTO:
#    - Selecciona la clase con mayor probabilidad
#    - Mapea índice a etiqueta: 0→NEG, 1→NEU, 2→POS
#    - Formatea como diccionario: {'label': 'POS', 'score': 0.95}
#
###############################################################################

# Clasificamos cada frase
# Este llamado ejecuta TODO el proceso descrito arriba
resultados = clasificador(frases)

# ¿Qué contiene 'resultados'?
# Una lista de diccionarios, uno por cada frase:
# [
#   {'label': 'POS', 'score': 0.9523},
#   {'label': 'NEG', 'score': 0.9876},
#   ...
# ]

# Mostramos los resultados
for i, (frase, resultado) in enumerate(zip(frases, resultados), 1):
    sentimiento = resultado['label']
    confianza = resultado['score']
    
    print(f"\nFrase {i}: '{frase}'")
    print(f"  Predicción: {sentimiento} (confianza: {confianza:.2f})")
    
    # Indicador de confianza
    if confianza >= 0.9:
        print(f"  Confianza: Muy alta")
    elif confianza >= 0.7:
        print(f"  Confianza: Alta")
    elif confianza >= 0.5:
        print(f"  Confianza: Media")
    else:
        print(f"  Confianza: Baja (poco seguro)")

print("\n" + "="*70)

###############################################################################
# Comparación: LSTM vs BETO
###############################################################################
print("\n¿Por qué BETO funciona mejor que nuestra LSTM?")
print("\n1. Arquitectura Transformer vs LSTM:")
print("   LSTM: Procesa secuencialmente (izquierda → derecha)")
print("         Puede 'olvidar' información del inicio")
print("   BETO: Self-attention mira TODA la frase simultáneamente")
print("         Cada palabra puede 'ver' todas las demás")
print("\n2. Contexto bidireccional:")
print("   LSTM: Solo ve palabras anteriores al procesar cada token")
print("   BETO: Ve contexto completo (antes y después) para cada token")
print("\n3. Capacidad del modelo:")
print("   LSTM (nuestro): 7,000 parámetros, entrenado con 10 frases")
print("   BETO: 110,000,000 parámetros, entrenado con millones de textos")
print("\n4. Vocabulario:")
print("   LSTM (nuestro): ~50 palabras únicas")
print("   BETO: ~31,000 tokens (incluyendo sub-palabras)")
print("\n5. Embeddings:")
print("   LSTM (nuestro): 16 dimensiones, aprendidos de 10 frases")
print("   BETO: 768 dimensiones, aprendidos de Wikipedia completa")
print("\n" + "="*70)

## 6. Análisis de expresiones rioplatenses

Vamos a probar específicamente con expresiones típicas de Argentina para ver si BETO las entiende.

In [ ]:
expresiones_argentinas = [
    "Está re copado este lugar",
    "Qué garrón, me clavaron",
    "La pasé bomba, volvería sin dudarlo",
    "Un bodrio total, no lo banco más",
    "Me re sirvió, gracias",
    "Qué embole, fue un bajón"
]

print("="*70)
print("BETO CON EXPRESIONES RIOPLATENSES")
print("="*70)

resultados = clasificador(expresiones_argentinas)

for frase, resultado in zip(expresiones_argentinas, resultados):
    print(f"\nFrase: '{frase}'")
    print(f"  {resultado['label']} ({resultado['score']:.2f})")

print("\n" + "="*70)
print("Observá cómo BETO entiende modismos argentinos como:")
print("  'copado', 'garrón', 'bomba', 'bodrio', 'embole', 'bajón'")
print("Esto es gracias a su entrenamiento con textos en español variados.")
print("="*70)

## 7. Casos desafiantes: Opiniones mixtas y matices

Probemos con casos más complejos que suelen confundir a modelos simples.

In [ ]:
casos_complejos = [
    "La comida estuvo bien pero el servicio fue horrible",
    "No estuvo mal, aunque esperaba más por el precio",
    "Pensé que iba a ser un desastre pero me sorprendió para bien",
    "Excelente producto, lástima la demora en el envío",
    "No puedo decir que no me gustó, porque sería mentir"
]

print("="*70)
print("CASOS COMPLEJOS: OPINIONES MIXTAS Y MATICES")
print("="*70)

resultados = clasificador(casos_complejos)

for i, (frase, resultado) in enumerate(zip(casos_complejos, resultados), 1):
    print(f"\nCaso {i}: '{frase}'")
    print(f"  Predicción: {resultado['label']} (confianza: {resultado['score']:.2f})")
    
    # Análisis del caso
    if i == 1:
        print(f"  Análisis: Opinión mixta (comida bien, servicio mal)")
    elif i == 2:
        print(f"  Análisis: Sentimiento tibio con matices")
    elif i == 3:
        print(f"  Análisis: Giro de expectativa negativa a positiva")
    elif i == 4:
        print(f"  Análisis: Positivo con salvedad")
    elif i == 5:
        print(f"  Análisis: Doble negación (no... no = sí)")

print("\n" + "="*70)

## 8. Comparación directa: ¿Cómo se compara con LSTM?

Recordemos que con LSTM entrenamos con 10 frases. BETO, en cambio, fue entrenado con millones de textos.

### Ventajas observadas de BETO:

1. **Vocabulario extenso**: Entiende palabras que nunca vio en nuestro pequeño dataset
2. **Contexto bidireccional**: Mira toda la frase simultáneamente, no solo de izquierda a derecha
3. **Modismos y expresiones**: Reconoce jerga argentina sin entrenamiento específico
4. **Matices**: Maneja mejor opiniones mixtas y giros en el sentimiento
5. **Confianza calibrada**: Los scores reflejan mejor la certeza del modelo

## 9. Probá tus propias frases

Ahora te toca experimentar. Agregá frases propias y observá cómo se comporta BETO.

In [ ]:
# Agregá tus propias frases acá
mis_frases = [
    "Tu frase 1",
    "Tu frase 2",
    "Tu frase 3"
]

# Descomentá estas líneas cuando tengas tus frases
# resultados_propios = clasificador(mis_frases)
# for frase, resultado in zip(mis_frases, resultados_propios):
#     print(f"\nFrase: '{frase}'")
#     print(f"  Predicción: {resultado['label']} ({resultado['score']:.2f})")

## 10. Reflexión final

### ¿Qué aprendimos?

1. **Transfer learning es poderoso**: No necesitamos entrenar desde cero para obtener buenos resultados en NLP.

2. **Modelos preentrenados en español**: Existen modelos como BETO que entienden texto coloquial, incluso modismos regionales.

3. **Facilidad de uso**: Con HuggingFace Transformers podemos usar modelos de estado del arte con apenas unas líneas de código.

4. **Escala importa**: Un modelo entrenado con millones de ejemplos es cualitativamente diferente a uno entrenado con 10.

### Comparación con nuestros modelos anteriores:

**Perceptrón (Lab 1)**:
- Entrenamiento: 6 frases, segundos
- Limitación: Modelo lineal, bag-of-words
- Resultado: Clasificación básica

**MLP (Lab 2)**:
- Entrenamiento: 10 frases, segundos
- Mejora: Patrones no lineales
- Limitación: Sigue sin orden, bag-of-words

**LSTM (Lab 3)**:
- Entrenamiento: 10 frases, minutos
- Mejora: Procesa secuencias, embeddings
- Limitación: Dataset pequeño, vocabulario limitado

**BETO (Lab 4)**:
- Entrenamiento: Millones de textos, días en GPUs
- Mejora: Comprensión profunda del español
- Uso: Sin entrenar, solo inferencia

### ¿Por qué esto cambió el NLP?

Antes de 2018 (pre-BERT), cada tarea requería:
1. Conseguir dataset grande específico
2. Entrenar modelo desde cero
3. Esperar que generalizara

Después de 2018 (era Transformer):
1. Usar modelo preentrenado
2. Opcionalmente: ajustar con pocos ejemplos (fine-tuning)
3. Obtener resultados superiores

### Conceptos clave para entender LLMs:

Lo que vimos hoy es la base de cómo funcionan los LLMs modernos:

- **Preentrenamiento masivo**: GPT, BERT, LLaMA se entrenan con billones de palabras
- **Arquitectura Transformer**: Atención en lugar de procesamiento secuencial
- **Transfer learning**: El conocimiento se transfiere entre tareas
- **Emergencia de capacidades**: Con suficiente escala, surgen habilidades no programadas explícitamente

### ¿Qué sigue?

En las próximas clases vamos a profundizar en:
- Arquitectura Transformer en detalle
- Mecanismo de atención
- Cómo se entrenan modelos como GPT
- Fine-tuning y prompting
- Construcción de aplicaciones con LLMs

Hoy vimos **qué** pueden hacer estos modelos. Próximamente veremos **cómo** funcionan por dentro.